In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import telebot #Biblioteca de criação do Bot Telegram
from time import sleep 
from os import walk, path
from datetime import date
from string import punctuation
from ipynb.fs.full.chamar import init
from ipynb.fs.full.xml2pdf_semforn import iniciapdf
from dateutil.relativedelta import relativedelta

from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton #Biblioteca dos botões e markup

meses = []
nome_meses = ['JANEIRO', 'FEVEREIRO', 'MARÇO', 'ABRIL', 'MAIO', 'JUNHO', 'JULHO', 'AGOSTO', 'SETEMBRO', 'OUTUBRO', 'NOVEMBRO', 'DEZEMBRO']
now = date.today()

for i in range(3,-1,-1):
    x = relativedelta(months=i)
    meses.append({nome_meses[((now - x).month)-1]:(now - x).month})
    
def junta_dici(ld):
    '''Função pra juntar uma lista de dicionários em um único'''
    resultado = {}
    for dici in ld:
        resultado.update(dici)
    return resultado

def junta_loja(pessoa):
    '''Função pra juntar as lojas contidas em um dicionário em uma única lista'''
    ljs = []
    print('entrei junta_loja')
    if pessoa['lj1']:
        ljs.append('0')
    if pessoa['lj2']:
        ljs.append('1')
    if pessoa['lj3']:
        ljs.append('2') 
    if pessoa['lj4']:
        ljs.append('3')
    if pessoa['lj5']:
        ljs.append('4')
    if pessoa['lj6']:
        ljs.append('5')
    if pessoa['lj7']:
        ljs.append('6')
    print('sai junta')
    return ljs
    
meses_junto = junta_dici(meses)
lis_dici = [{'ID':'0','FORN':'','lj1':False,'lj2':False,'lj3':False,'lj4':False,'lj5':False,'lj6':False,'lj7':False,'pdf':False, 'mes':now.month}]

def insere_ld(id_,item, mes=0, forn=0):
    '''Função pra inserir algo nas escolhas do usuário'''
    global lis_dici                  #Pega o dict de escolhas pelo global
    for i in lis_dici:
        if i['ID'] == id_:
            if item == 'pdf':
                i['pdf'] = True
                return True
            if item ==  'preconf':
                i['pdf'] = False
                return True
            if not mes and not forn: #Se não mes e forn, o item mandado vira True
                i[item] = True
                return True
            elif mes:                #Se mes, declara o mes de preferencia dentro do dict de escolhas
                i['mes'] = item
                return True
            elif forn:              #Se forn, declara o forn de preferencia dentro do dict de escolhas
                i['FORN'] = item
                return True
    lis_dici.append({'ID':id_,'lj1':False,'lj2':False,'lj3':False,'lj4':False,'lj5':False,'lj6':False,'lj7':False,'pdf':False, 'mes':now.month}) #Se não achar o ID ele cria um
    insere_ld(id_, item, mes, forn) #E manda inserir oq foi pedido dentro do id criado

    
def limpa_ld(id_):
    '''zera as escolhas que não forem ID, MES ou FORN'''
    global lis_dici
    for i in lis_dici:
        if i['ID'] == id_:
            for k,v in i.items():
                if k != 'ID' and k != 'mes' and k != 'FORN':
                    i[k] = False
    

def pega_escolhas(id_):
    '''Função pra retornar o dict de escolhas a partir de um ID'''
    global lis_dici
    for i in lis_dici:
        if i['ID'] == id_:
            return i
    
    
def uma_linha(nome_arquivo):
    '''Função pra abrir arquivos de texto e retornar a primeira linha'''
    with open (f'C:/Users/pdv/Desktop/gabriel/{nome_arquivo}', 'r') as arquivo:
        for linha in arquivo:
            return linha        

            
chaveapi = uma_linha('chaveapitelebot.txt')    #Pega a chave API do bot de um arquivo local  
bot = telebot.TeleBot(chaveapi) #Cria uma instância do bot


def checkforn(forn):
    '''Função que devolve uma lista de fornecedores (e cnpjs) a partir de uma string'''
    cnpjs, nomes, cnpj = [], [], ''
    with open("cnpjs.txt", 'r') as cads:
        for linha in cads:
            if forn.lower() in linha.lower():
                cnpj = ''.join([i for i in linha[-19:-9] if i not in punctuation])
                cnpjs.append(cnpj)
                nomes.append(linha[0:-19])
        return (cnpjs, nomes)
            
            
def faz_botao(dicionario): 
    '''Função pra criar os botões de resposta'''
    markup = InlineKeyboardMarkup() #inicia o markup (marcação)
    for botao, retorno in dicionario.items(): #Desempacota os botões e seus retornos do dicionário
        markup.add(InlineKeyboardButton(str(botao), callback_data=str(retorno))) #cria o botão dentro do markup
    return markup #retorna o markup
    
    
def poezero(num):
    '''Função pra pegar um mes e se for menor que 10 coloca o zero'''
    try:
        num = int(num)
    except:
        print('erro no poezero')
    mes = ''
    if num <= 9:
        mes = f'0{str(num)}'
    else:
        mes = str(num)
    return mes
    

def envia(mensagemID, pdf = 0):
    pasta = fr'C:\Users\pdv\projetos_jupyter\retorno\\{mensagemID}'
    if pdf:
        pasta = fr'C:\Users\pdv\Desktop\selenium-imgs\pdfs\\{mensagemID}'
    for diretorio, subpastas, arquivos in walk(pasta):
        print('for do envia')
        if not arquivos:
            text = 'Desculpe, não achei notas desse fornecedor nessa loja'
            bot.send_message(mensagemID,text)
            return 0
        for arquivo in arquivos:
            x = path.join(pasta, arquivo)
            print(x, 'tentei enviar')
            if pdf:
                print('to in')
                file = open(x, 'rb')
                bot.send_document(mensagemID, file)
                file.close()
            else:
                photo = open(x,'rb')
                bot.send_photo(mensagemID, photo=photo)
                photo.close()
    bot.reply_to(mensagem, 'pronto ;)') #Mensagem pra mostrar que deu certo

    
    
@bot.message_handler(commands=["notas"])
def notas(mensagem):
    '''Função que gerencia o /notas'''
    global meses_junto
    id_ = mensagem.chat.id #O id é uma parte da estrutura aninhada da mensagem ({'content_type': 'text', 'id': 9, 'message_id': 9,'from_user': {'id': 945441763, 'is_bot': False, 'first_name': 'Gbr', 'username': 'Gbrdoidao', 'last_name': 'Doidao', 'language_code': 'pt-br', 'can_join_groups': None, 'can_read_all_group_messages': None, 'supports_inline_queries': None}, 'date': 1649940793,'chat': {'id': 945441763, 'type': 'private', 'title': None, 'username': 'Gbrdoidao', 'first_name': 'Gbr', 'last_name': 'Doidao', 'photo': None, 'bio': None, 'description': None, 'invite_link': None, 'pinned_message': None, 'permissions': None, 'slow_mode_delay': None, 'message_auto_delete_time': None, 'sticker_set_name': None, 'can_set_sticker_set': None, 'linked_chat_id': None, 'location': None},'sender_chat': None, 'forward_from': None, 'forward_from_chat': None, 'forward_from_message_id': None, 'forward_signature': None, 'forward_sender_name': None, 'forward_date': None, 'reply_to_message': None, 'via_bot': None, 'edit_date': None, 'media_group_id': None, 'author_signature': None, 'text': 'ao', 'entities': None, 'caption_entities': None, 'audio': None, 'document': None, 'photo': None, 'sticker': None, 'video': None, 'video_note': None, 'voice': None, 'caption': None, 'contact': None, 'location': None, 'venue': None, 'animation': None, 'dice': None, 'new_chat_member': None, 'new_chat_members': None, 'left_chat_member': None, 'new_chat_title': None, 'new_chat_photo': None, 'delete_chat_photo': None, 'group_chat_created': None, 'supergroup_chat_created': None, 'channel_chat_created': None, 'migrate_to_chat_id': None, 'migrate_from_chat_id': None, 'pinned_message': None, 'invoice': None, 'successful_payment': None, 'connected_website': None, 'reply_markup': None, 'json': {'message_id': 9, 'from': {'id': 945441763, 'is_bot': False, 'first_name': 'Gbr', 'last_name': 'Doidao', 'username': 'Gbrdoidao', 'language_code': 'pt-br'}, 'chat': {'id': 945441763, 'first_name': 'Gbr', 'last_name': 'Doidao', 'username': 'Gbrdoidao', 'type': 'private'}, 'date': 1649940793, 'text': 'ao'}})
    enviado = mensagem.text[7:] #Pega a parte importante da mensagem (sem o /notas )
    fornecedor = checkforn(enviado)
    print('resultado checkforn: ', fornecedor)
    if not fornecedor[0]:
        bot.reply_to(mensagem, 'Fornecedor nâo encontrado') #Mensagem pra mostrar que deu errado a busca de forn
        return 0
    if len(fornecedor[0]) > 1:
        dicio_forn = {}
        for cnpj in fornecedor[0]:
            for empresa in fornecedor[1]:   
                dicio_forn[empresa] = f'forn: {cnpj}'
        print(dicio_forn)
        bot.send_message(id_, 'Qual Fornecedor?', reply_markup=faz_botao(dicio_forn))
    else:
        insere_ld(id_, fornecedor[0][0], forn=1) #Coloca o fornecedor em escolhas (dicionario do user)

    print(mensagem.from_user.username,' pesquisou notas do ', fornecedor)
    dicio_pdf = {'Preço Notas':'preconf','Nota em PDF':'pdf'} #Botão pdf / preço {'nome_botão':'retorno do botão'}
    dicio_lojas = {'LJ1 - Guanabara' : 'lj1' ,'LJ2 - Primavera' : 'lj2','LJ3 - Iguatemi' : 'lj3','LJ4 - Flamboyant' : 'lj4', 'LJ5 - São Paulo' : 'lj5','LJ6 - Valinhos' : 'lj6',
                   'LJ7 - Ouro Verde' : 'lj7', 'ESCOLHIDO' : 'escolheu'} #Botões das lojas e seus retornos 
    bot.send_message(id_, ' Qual Mes? ', reply_markup=faz_botao(meses_junto)) #Botao pro user escolher o mes da nota
    bot.send_message(id_, ' Qual vc prefere? ' ,reply_markup=faz_botao(dicio_pdf)) #Cria o botão pro escolher entre PDF e meu preço
    bot.send_message(id_, ' LOJAS: ' ,reply_markup=faz_botao(dicio_lojas)) #Cria o botão pro escolher entre PDF e meu preço
    

@bot.message_handler(func=lambda message: True) #Decorador que cuida de mensagens (não comandos)
def resposta(mensagem):
    texto = '''Olá! Seja bem vindo ao Fartura Bot  
    Estes são os meus Comandos:
    /notas <Nome Fornecedor>
    =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=''' #Mensagem padrão de resposta
    bot.reply_to(mensagem, texto) #Responde a mensagem dele com o texto padrão
    
    
@bot.callback_query_handler(func=lambda call: True)
def handle_query(chamado):
    nome = chamado.message.from_user.username
    id_ = chamado.message.chat.id
    retorno = chamado.data
    inteiro = False
    try:
        inteiro = int(retorno)
    except:
        pass
    if retorno.startswith('lj') or retorno == 'pdf':
        insere_ld(id_,retorno)
    elif inteiro and inteiro in range(13): #Pega o retorno dos botões de meses
        insere_ld(id_, retorno, mes=1)
    elif retorno == 'escolheu':
        print('escolheu')
        escolhas = pega_escolhas(id_)
        print('pegou escolhas')
        lojas = junta_loja(escolhas)
        print('pegou lojas', escolhas['mes'])
        mes = poezero(escolhas['mes'])
        pdf = escolhas['pdf']
        forn = escolhas['FORN']
        print('forn: ', forn, type(forn))
        if pdf:
            print('vou chamar inicia pdf')
            print(id_, type(id_) , forn, type(forn), 0, 'int' ,lojas,type(lojas),  mes, type(mes))
            r = iniciapdf(id1 = id_, fornecedor = forn, numero = 0, lojas = lojas, mes = mes)
            print('vou chamar enviapdf')
            envia(id_, pdf=1)
            print('pdf enviado')
        else:
            print('vou chamar init')
            init(ean = forn, mes = mes, loja = lojas, idz = id_)
            print('vou chamar envia init')
            envia(id_, pdf=0)
            print('init enviado')
        limpa_ld(id_)
    elif retorno.startswith('forn:'):
        insere_ld(id_, retorno, forn=1)
    
    
while True:
    try:
        bot.polling(none_stop=True, interval=0, timeout=0) #Mantém o bot rodando infinitamente
    except:
        sleep(10) #se der erro ele espera 10 segundos


In [ ]:
'''import time

def calcula_tempo(funcao):
    def wrapper(*args, **kwargs):     
        tempo_comeco = time.time()
        x = funcao(*args, **kwargs)
        tempo_final = time.time()
        print(f'sua função demorou {tempo_final - tempo_comeco}')
        return x
    return wrapper
        
        
l1 = list(range(-200,4000))
l2 = list(range(3000))
l3 = []

@calcula_tempo
def checa_lista(l1,l2,l3):
    for i in l1:
        if i not in l2:
            l3.append(i)
    return l3

print(checa_lista(l1,l2,l3))

@calcula_tempo
def Diff(l1, l2):
    return list(set(l1) - set(l2)) + list(set(l2) - set(l1))

print(Diff(l1,l2))'''

In [ ]:
linha = 'abcdefghijklmnopqrstuvwxyz'
print(linha[0:-19])

In [ ]:
forn = (['75315333', '01109364', '09323835', '00286330', '49092398', '35835486', '08142803', '39341518', '32619152', '67765842', '20977899', '05868574', '01157555'], ['ATACADAO S.A. -003671\t', 'BATAVO LLOC -002644\t', 'BTWO COMPANY BRASIL ATACADO E DIST. LTDA -005227\t', 'CATABY FILIAL CD -002668\t', 'COMERCIAL ESPERANCA ATACADO DISTRIBUIDOR LTDA -001258\t', 'CORUJAO ATACAREJO POTIGUAR -005349\t', 'CRISTINA QUATA/NOVA MIX INDL COM ALIMENTOS -001856\t', 'HRC E LA BRASA COMERCIO ATACADISTA DE ALIMENTOS LTDA -005026\t', 'LUCROCS COMERCIO ATACADISTA DE AMENDOIM LTDA -004598\t', 'MAURO YOSHIO NAKATA EIRELI -004080\t', 'MM SP DISTRIB E COM ATACADISTA LTDA -002964\t', 'SPANI ATACADISTA ZARAGOZA -001514 \t', 'TENDA ATACADOS '])
for i in forn[0]:
    for j in forn[1]:
        print(i,j)